In [1]:
from selenium import webdriver
from concurrent.futures import ThreadPoolExecutor
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from threading import Lock
import time
import json
import re
import os

In [ ]:
# Configurar o WebDriver (sem headless)
chrome_options = Options()

service = Service("/opt/homebrew/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=chrome_options)

# Acessar o endpoint
driver.get("https://www."substituir".com.br/api/catalog_system/pub/category/tree/3") 

# Garantir que a página está completamente carregada
WebDriverWait(driver, 10).until(
    lambda d: d.execute_script('return document.readyState') == 'complete'
)

# Aguardar até que o elemento <pre> apareça
try:
    pre_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, "//pre"))
    )
    json_text = pre_element.text  # Recuperar o texto dentro de <pre>

    # Converter o texto JSON em uma lista (array)
    data_array = json.loads(json_text)

    # Exibir o conteúdo do array
    print(data_array)

    # Salvar o array em um arquivo JSON
    with open("../../data/raw/dados.json", "w", encoding="utf-8") as json_file:
        json.dump(data_array, json_file, ensure_ascii=False, indent=4)  # Salvando o arquivo com indentação

    print("Dados salvos no arquivo '/data/processed/dados.json'.")

except Exception as e:
    print(f"Erro ao localizar o elemento: {e}")

# Fechar o WebDriver
driver.quit()


In [ ]:
# Exemplo do seu array JSON carregado (substitua pelo seu array real)
with open('../../data/raw/dados.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Função recursiva para extrair apenas os links das categorias mais específicas (sem filhos)
def extract_leaf_links(data):
    leaf_links = []

    # Verificar se o objeto atual é um dicionário
    if isinstance(data, dict):
        # Verificar se tem a chave 'url' e se 'children' está vazio ou não existe
        if 'url' in data and ('children' not in data or not data['children']):
            leaf_links.append(data['url'])  # Adicionar o link da categoria mais específica
            print(f"Link mais específico encontrado: {data['url']}")  # Exibir o link encontrado para depuração

        # Se tiver filhos em 'children', processar recursivamente
        elif 'children' in data and isinstance(data['children'], list):
            for child in data['children']:
                leaf_links.extend(extract_leaf_links(child))  # Recursão para os filhos
    
    # Verificar se o objeto atual é uma lista
    elif isinstance(data, list):
        for item in data:
            leaf_links.extend(extract_leaf_links(item))  # Processar cada item da lista

    return leaf_links

# Extrair apenas os links das categorias mais específicas (sem filhos)
leaf_links = extract_leaf_links(data)

# Verificar se os links foram extraídos corretamente
print(f"Total de links mais específicos extraídos: {len(leaf_links)}")
print(leaf_links)

# Salvar os links extraídos em um arquivo
with open('../../data/raw/links_extrated.json', 'w', encoding='utf-8') as output_file:
    json.dump(leaf_links, output_file, ensure_ascii=False, indent=4)

print("Links salvos em 'data/raw/links_extrated.json'.")


In [ ]:
# Lista global para acumular todos os skuIds
all_sku_ids = []
# Caminhos dos arquivos
links_extrated = "../../data/raw/links_extrated.json"
processed_links_array = "../../data/raw/processed_links.json"
products_ids = "../../data/raw/products_ids_list.json"

# Mutex para controlar o acesso à lista global em threads
lock = Lock()

# Função para configurar o WebDriver e processar um link com paginação e retry
def process_link(link, max_retries=3):
    attempt = 0
    while attempt < max_retries:
        chrome_options = Options()
        service = Service("/opt/homebrew/bin/chromedriver")
        driver = webdriver.Chrome(service=service, options=chrome_options)

        try:
            driver.get(link)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.TAG_NAME, "script"))
            )

            # Verificação de mensagem "Ops, não encontramos nada em sua busca"
            if "Ops, não encontramos nada em sua busca" in driver.page_source:
                print(f"Nenhum produto encontrado para o link: {link}")
                # Adicionar o link diretamente ao arquivo de links processados
                with lock:
                    if not os.path.exists(processed_links):
                        with open(processed_links, "w") as f:
                            json.dump([], f)
                    with open(processed_links, "r") as f:
                        processed_links = json.load(f)
                    processed_links.append(link)
                    with open(processed_links, "w") as f:
                        json.dump(processed_links, f, indent=4)
                return  # Sair da função, pois não há produtos para processar

            # Verificação de paginação e processamento dos produtos
            page_numbers = driver.find_elements(By.CSS_SELECTOR, "ul.pages li.page-number")
            total_pages = int(page_numbers[-1].text) if page_numbers else 1

            link_processed_successfully = False  # Indicador de sucesso para o link

            for page in range(1, total_pages + 1):
                paginated_link = f"{link}#{page}"
                driver.get(paginated_link)
                time.sleep(2)

                scripts = driver.find_elements(By.TAG_NAME, "script")

                for script in scripts:
                    script_content = script.get_attribute("innerHTML")

                    if '"shelfProductIds"' in script_content:
                        match = re.search(r'"shelfProductIds":\[(.*?)\]', script_content)
                        if match:
                            shelf_product_ids = [sku.strip('"') for sku in match.group(1).split(',')]
                            with lock:
                                all_sku_ids.extend(shelf_product_ids)
                            print(f"Link: {paginated_link} - Página {page} - shelfProductIds: {shelf_product_ids}")
                            link_processed_successfully = True
                        break

            # Registrar o link como processado apenas se algum `skuID` foi coletado
            if link_processed_successfully:
                with lock:
                    if not os.path.exists(processed_links_array):
                        with open(processed_links_array, "w") as f:
                            json.dump([], f)
                    with open(processed_links_array, "r") as f:
                        processed_links_array = json.load(f)
                    processed_links_array.append(link)
                    with open(processed_links_array, "w") as f:
                        json.dump(processed_links_array, f, indent=4)

            return

        except Exception as e:
            print(f"Erro ao processar o link {link} na tentativa {attempt + 1}: {e}")
            attempt += 1
            time.sleep(2)
        
        finally:
            driver.quit()

    print(f"Falha ao processar o link {link} após {max_retries} tentativas.")

# Função para verificar e tentar reprocessar links faltantes até que todos sejam processados
def retry_missing_links():
    with open(links_extrated, 'r', encoding='utf-8') as f:
        all_links = set(json.load(f))

    while True:
        with open(processed_links_array, 'r') as f:
            processed_links = set(json.load(f))

        # Identificar links que não foram processados
        remaining_links = list(all_links - processed_links)
        if not remaining_links:
            break

        print(f"{len(remaining_links)} links não foram processados. Tentando novamente.")

        # Reprocessar os links restantes
        with ThreadPoolExecutor(max_workers=6) as executor:
            executor.map(process_link, remaining_links)

# Executar a primeira rodada de scraping
with open(links_extrated, 'r', encoding='utf-8') as f:
    links = json.load(f)

# Executar as requisições em paralelo com ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=6) as executor:
    executor.map(process_link, links)

# Tentar novamente os links que não foram processados com sucesso
retry_missing_links()

# Salvar todos os skuIds extraídos em um arquivo JSON
with open(products_ids, 'w', encoding='utf-8') as output_file:
    json.dump(all_sku_ids, output_file, ensure_ascii=False, indent=4)

print("Todos os skuIds foram salvos no arquivo 'lista_skuID_produtos.json'.")